<a href="https://colab.research.google.com/github/GerardoToso/Competencia-FSadosky-e-YPF-/blob/main/02_Baseline_toda_la_poblacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">  
           <img 
               src="" 
               style="width:10px"
           >
           </a>    
       </td>
   </tr>
</table>
<div style="float:right;"><h2>02. Baseline toda la población</h2></div>

 <a id="top"></a> <br>
## Índice
1. [Entradas y Entregable](#1)<br>
    1.1. [Entradas del modelo](#11)<br>
    1.2. [Entregable](#12)<br>
2. [Carga de paquetes y datos](#2)<br>
    2.1. [Librerías y clases](#21)<br>
    2.2. [Lectura de datos](#22)<br>
    2.3. [Eliminación de valores nulos](#23)<br>
    2.4. [Separación en train y test](#24)<br>
3. [Modelo](#3)<br>
    3.1. [Confección pipeline](#31)<br>
    3.2. [Optimización de hiperparámetros](#32)<br>
    3.3. [Visualizaciones de hiperparámetros](#33)<br>
    3.4. [Entrenamiento del modelo con mejores hiperparámetros](#34)<br>
4. [Evaluación del modelo](#4)<br>
    4.1. [Cálculo de métricas](#41)<br>
    4.2. [Matriz de confusión](#42)<br>
    4.3. [ROC Curve](#43)<br>
    4.4. [Precision-Recall Curve](#44)<br>
    4.5. [Feature importance](#45)<br>
5. [Exportación con shelve](#5)<br>
    5.1. [Exportación con shelve](#51)<br>

***
<a id="1"></a> <br>
## 1. Entradas y Entregable

<a id="11"></a>
### 1.1. Entradas

|Tipo|Nombre|Descripción|Ubicación Lab|
|:-------|----------------|---------------------------------------------------|-------------------------------------|
|**csv**| `Dataset participantes.csv` | datos relacionados con los pozos al producirse una interferencia por la estimulación de un pozo hijo |`/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/`|

<a id="12"></a>
### 1.2. Entregable

**Archivo csv con las estimaciones de delta_WHP**

***
<a id="2"></a> <br>
## 2. Carga de paquetes y datos

<a id="21"></a>
### 2.1. Librerías y clases

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib. pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm as lgb

import chardet
%matplotlib inline

from IPython.display import Image
import pydotplus

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Esta clase permite dropear columnas de un DataFrame dentro de un pipeline

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self  

<a id="22"></a> <br>
### 2.2. Lectura de datos

In [ ]:
# Detección automática de la codificación de los caracteres
with open("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes.csv", 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

# Lectura del csv
df = pd.read_csv("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes.csv", encoding = enc['encoding'],sep='\t', index_col=0)

df.sample(5)

In [ ]:
# Se cambia el tipo de datos de variables categóricas nominales de object a category
df = df.astype({"CAMPO": "category", "FLUIDO": "category", "PAD_HIJO": "category", "HIJO": "category",
                "ETAPA_HIJO": "category", "PADRE": "category", "LINEAMIENTO": "category", "ESTADO": "category"})
df.dtypes

<a id="23"></a> <br>
### 2.3. Eliminación de valores nulos
**Dado que el modelo a desarrollar en el presente notebook se tomará como baseline, no se incluyen en la solución los valores nulos**

In [ ]:
print("Cantidad de instancias previo a la eliminación de valores nulos: ", df.shape[0])
df.dropna(inplace= True)
print("Cantidad de instancias posterior a la eliminación de valores nulos: ", df.shape[0])

<a id="24"></a> <br>
### 2.4. Separación en train y test

In [ ]:
# Selección de variable target
y = df.delta_WHP
# Selección de features
X = df.drop(["delta_WHP"], axis = 1)

# Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 161)

# Verificación de distribución de variable targuet en train y test
print("Instancias de train \n", X_train.shape[0])
print("Instancias de test \n", X_test.shape[0])

***
<a id="#3"></a> <br>
## 3. Modelo

<a id="31"></a> <br>
### 3.1. Confección pipeline

In [ ]:
# Instanciado de estimador
light_gbm = lgb.LGBMRegressor(random_state = 45)

# Instanciado de pipeline
pipe = Pipeline([ ("ColumnDropper", ColumnDropper(["ID_EVENTO"])),
                  
                  ("column_transformer",ColumnTransformer([
                                        ("standard_scaler", StandardScaler(), 
                    ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                 
                                 ("light_gbm", light_gbm)])

<a id="32"></a> <br>
### 3.2. Optimización de hiperparámetros

In [ ]:
# Optimización de hiperparámetros del modelo con grid search y cross validation

kfolds=KFold(n_splits=3,shuffle=True, random_state=42)

params_grid = { 
                "light_gbm__max_depth" : [5,10,50,80],
                "light_gbm__learning_rate": [0.01,0.05,1],
                "light_gbm__n_estimators" : [5,100,500,1000] 
                              }

model = GridSearchCV (pipe, params_grid, cv=kfolds, n_jobs = -1, scoring = "neg_root_mean_squared_error", refit = True)

In [ ]:
# Seteado de parámetros de entrenamiento
fit_params={"light_gbm__categorical_feature": "auto" }

# Entrenamiento del modelo
model.fit(X_train,y_train, **fit_params)

In [ ]:
# Mejores hiperparámetros obtenidos por cross validation y grid search
model.best_params_

In [ ]:
# Valor de RMSE en train
print("RMSE en train = ",mean_squared_error(y_train, model.predict(X_train), squared=False))
print("RMSE en test = ",mean_squared_error(y_test, model.predict(X_test), squared=False))